In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math

import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm.auto import tqdm
import sys

import ddpm
import datasets

c:\Users\jgual\anaconda3\envs\your_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

print(f"PyTorch version Installed: {torch.__version__}\nTorchvision version Installed: {torchvision.__version__}\n")
if not torch.__version__.startswith("1.11"):
    print("you are using an another version of PyTorch. We expect PyTorch 1.11.0. You may continue using your version but it"
          " might cause dependency and compatibility issues.")
if not torchvision.__version__.startswith("0.12"):
    print("you are using an another version of torchvision. We expect torchvision 0.12. You can continue with your version but it"
          " might cause dependency and compatibility issues.")

PyTorch version Installed: 1.13.1
Torchvision version Installed: 0.14.1+cpu

you are using an another version of PyTorch. We expect PyTorch 1.11.0. You may continue using your version but it might cause dependency and compatibility issues.
you are using an another version of torchvision. We expect torchvision 0.12. You can continue with your version but it might cause dependency and compatibility issues.


**DATASET & DATALOADER**

In [4]:
# DATA NORMALIZATION PRIOR TO DATASET GENERATION

from grasp_object_dataset import graspDataset
from torch.utils.data import DataLoader

def get_mean_std(main_dir, object_dir, dataset):

    # Allocate variables
    full_joints = []

    # Load all samples - only joint values!
    for sample in tqdm(dataset):
        joints = sample[0]
        full_joints.append(joints)

    mean = np.mean(full_joints, axis = 0)
    std = np.std(full_joints, axis = 0)
    max = np.max(full_joints, axis = 0)
    min = np.min(full_joints, axis = 0)
    

    return mean, std, max, min


In [11]:
# DATASET AND DATALOADER

from grasp_object_dataset import graspDataset
from torch.utils.data import DataLoader

main_dir = './dataset_grasps/'
object_dir = './dataset_objects/'

# Generate dataset with all dataset samples
main_dataset = graspDataset(main_dir, object_dir, mode = 'train', split = {'train': 1, 'val': 0, 'test': 0}, normalization=None, transform_joint = None, transform_object = None)

mean_std_max_min = list(get_mean_std(main_dir, object_dir, main_dataset))
print('NOT NORMALIZED: ', mean_std_max_min)


'''
transform_joint = transforms.Compose([
    # Apply any necessary transformations
])

transform_object = transforms.Compose([
    # Apply any necessary transformations
])
'''

train_dataset = graspDataset(main_dir, object_dir, mode = 'train', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)
val_dataset = graspDataset(main_dir, object_dir, mode = 'val', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)
test_dataset = graspDataset(main_dir, object_dir, mode = 'test', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)

'''
mean_std_max_min = list(get_mean_std(main_dir, object_dir, train_dataset))
print('NORMALIZED: ', mean_std_max_min)
'''

train_dataloader = DataLoader(train_dataset , batch_size=10, shuffle=False, num_workers=2, drop_last=False)
val_dataloader = DataLoader(val_dataset , batch_size=10, shuffle=False, num_workers=2, drop_last=False)
test_dataloader = DataLoader(test_dataset , batch_size=10, shuffle=False, num_workers=2, drop_last=False)

100%|██████████| 21000/21000 [00:40<00:00, 517.18it/s]


NOT NORMALIZED:  [array([ 3.98561844e-02,  2.57192661e-01,  5.95373810e-01,  1.67236000e-01,
        2.65023496e-02,  2.65105132e-01,  6.80795671e-01,  1.57267439e-01,
        1.17773209e-02,  3.03083575e-01,  6.61583456e-01,  1.70557048e-01,
        2.61156612e-01, -1.07516990e-01,  2.03640379e-01,  5.48817102e-01,
        2.04976352e-01,  2.73408066e-01,  1.04242190e+00, -1.19935514e-02,
       -2.36260475e-01, -6.06011368e-02, -6.05525100e-02,  8.50713967e-03,
       -2.46173353e-02,  6.38794029e-05, -3.58334739e-03, -7.04945023e-04]), array([0.08841094, 0.12425186, 0.13485115, 0.17028817, 0.08242855,
       0.10374933, 0.19098119, 0.17153302, 0.09209367, 0.11868084,
       0.17730865, 0.17665884, 0.10540849, 0.08868023, 0.09023743,
       0.23112383, 0.19124254, 0.22412766, 0.11790531, 0.10479109,
       0.20813474, 0.08463916, 1.78267916, 0.65887648, 1.78051084,
       0.09024645, 0.09977398, 0.08307887]), array([0.44093332, 0.72629118, 1.05269575, 0.85050714, 0.34213725,
       0

In [12]:
print(train_dataset[0][3]) # segundo indice --> 0: joints; 1: grasp label; 2: matrix distances

mujoco-Folic_Acid.npy


**DIFFUSION MODEL**

In [12]:
# HYPERPARAMETERS
# model MLP
hidden_size = 128
hidden_layers = 3
emb_size= 128
time_emb= "sinusoidal"
input_emb= "sinusoidal"
# Noise scheduler
num_timesteps = 50
beta_schedule= 'linear'
# optimizer
learning_rate = 1e-4

#training
num_epochs = 15

In [13]:
model = ddpm.MLP(
        hidden_size= hidden_size,
        hidden_layers=hidden_layers,
        emb_size= emb_size,
        time_emb= time_emb,
        #input_emb= input_emb
        )

noise_scheduler = ddpm.NoiseScheduler(
        num_timesteps=num_timesteps,
        beta_schedule=beta_schedule)

optimizer = torch.optim.AdamW(
        model.parameters(),
        lr= learning_rate,
    )

global_step = 0
frames = []
losses = []
print("Training model...")
for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(total=len(train_dataloader))
    progress_bar.set_description(f"Epoch {epoch}")
    for step, batch in enumerate(train_dataloader):
        # joints_angles = batch[0] // grasp_label = batch[1] // mat_distances = batch[2]
        joint_angles_batch = batch[0] 
        label_one_hot = batch[1]
        mat_distances = batch[2] 

        noise = torch.randn(joint_angles_batch.shape)
        timesteps = torch.randint(0, noise_scheduler.num_timesteps, (joint_angles_batch.shape[0],)).long()

        noisy = noise_scheduler.add_noise(joint_angles_batch, noise, timesteps)
        noisy = noisy.double()
        noise_pred = model(noisy, timesteps, label_one_hot, mat_distances)
        loss = F.mse_loss(noise_pred, noise)
        loss.backward(loss)

        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        logs = {"loss": loss.detach().item(), "step": global_step}
        losses.append(loss.detach().item())
        progress_bar.set_postfix(**logs)
        global_step += 1
    progress_bar.close()

"""
    if epoch % config.save_images_step == 0 or epoch == config.num_epochs - 1:
        # generate data with the model to later visualize the learning process
        model.eval()
        sample = torch.randn(config.eval_batch_size, 2)
        timesteps = list(range(len(noise_scheduler)))[::-1]
        for i, t in enumerate(tqdm(timesteps)):
            t = torch.from_numpy(np.repeat(t, config.eval_batch_size)).long()
            with torch.no_grad():
                residual = model(sample, t)
            sample = noise_scheduler.step(residual, t[0], sample)
        frames.append(sample.numpy())

print("Saving model...")
outdir = f"exps/{config.experiment_name}"
os.makedirs(outdir, exist_ok=True)
torch.save(model.state_dict(), f"{outdir}/model.pth")

print("Saving images...")
imgdir = f"{outdir}/images"
os.makedirs(imgdir, exist_ok=True)
frames = np.stack(frames)
xmin, xmax = -6, 6
ymin, ymax = -6, 6
for i, frame in enumerate(frames):
    plt.figure(figsize=(10, 10))
    plt.scatter(frame[:, 0], frame[:, 1])
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    plt.savefig(f"{imgdir}/{i:04}.png")
    plt.close()
    
print("Saving loss as numpy array...")
np.save(f"{outdir}/loss.npy", np.array(losses))
print("Saving frames...")
np.save(f"{outdir}/frames.npy", frames)
"""


Training model...


Epoch 0:   0%|          | 0/1680 [00:00<?, ?it/s]c:\Users\jgual\Desktop\BEMP\Advanced Deep Learning For Robotics\ADLR_project\tiny-diffusion\ddpm.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor= torch.tensor(label, dtype=torch.float32)
Epoch 14: 100%|██████████| 1680/1680 [19:22<00:00,  1.45it/s, loss=0.706, step=25199]


'\n    if epoch % config.save_images_step == 0 or epoch == config.num_epochs - 1:\n        # generate data with the model to later visualize the learning process\n        model.eval()\n        sample = torch.randn(config.eval_batch_size, 2)\n        timesteps = list(range(len(noise_scheduler)))[::-1]\n        for i, t in enumerate(tqdm(timesteps)):\n            t = torch.from_numpy(np.repeat(t, config.eval_batch_size)).long()\n            with torch.no_grad():\n                residual = model(sample, t)\n            sample = noise_scheduler.step(residual, t[0], sample)\n        frames.append(sample.numpy())\n\nprint("Saving model...")\noutdir = f"exps/{config.experiment_name}"\nos.makedirs(outdir, exist_ok=True)\ntorch.save(model.state_dict(), f"{outdir}/model.pth")\n\nprint("Saving images...")\nimgdir = f"{outdir}/images"\nos.makedirs(imgdir, exist_ok=True)\nframes = np.stack(frames)\nxmin, xmax = -6, 6\nymin, ymax = -6, 6\nfor i, frame in enumerate(frames):\n    plt.figure(figsize=(

MODEL INFERENCE

In [70]:
import os
import trimesh

main_dir = './dataset_grasps/'
object_dir = './dataset_objects/'

grasp_code = 'core-bottle-1ae823260851f7d9ea600d1a6d9f6e07'

matriz_distancias = np.load(os.path.join(object_dir+grasp_code+".npy"), allow_pickle=True)*0.11
matriz_distancias = torch.from_numpy(matriz_distancias)
matriz_distancias = matriz_distancias.reshape(1, 50, 50, 50)

label = torch.tensor([1, 0, 0])
label = label.reshape(1, 3)

In [71]:
model.eval()

eval_batch_size = 1
num_timesteps = 50
plot_step = 5

noise_scheduler = ddpm.NoiseScheduler(num_timesteps=num_timesteps)
sample = torch.randn(eval_batch_size, 28)

timesteps = list(range(num_timesteps))[::-1]

samples = []
steps = []

for i, t in enumerate(tqdm(timesteps)):
    t = torch.from_numpy(np.repeat(t, eval_batch_size)).long()
    with torch.no_grad():
        residual = model(sample, t, label, matriz_distancias)
        
    sample = noise_scheduler.step(residual, t[0], sample)
    if (i + 1) % plot_step == 0:
        samples.append(sample.numpy())
        steps.append(i + 1)

100%|██████████| 50/50 [00:03<00:00, 16.34it/s]


In [72]:
print(samples[-1][0])

[ 1.2512822   0.6747379   0.11242113 -0.5892519   1.3973362  -2.3288672
  2.330746   -0.43743408  0.62414175 -0.8976362   0.23073828 -0.5350091
 -0.15259601  0.61756146 -0.20531933 -0.20036286 -0.5556382  -0.837608
 -0.08550881 -1.3377986   0.33129486  1.8645691  -0.3787077  -2.1561317
 -0.9231335   1.3091593   0.5608413  -1.3097106 ]


In [ ]:
num_cols = 5
num_rows = math.ceil(len(samples) / num_cols)
fig = plt.figure(figsize=(15, 6))
for i, sample in enumerate(samples):
    plt.subplot(num_rows, num_cols, i + 1)
    plt.scatter(sample[:, 0], sample[:, 1], alpha=0.5, s=15)
    plt.title(f"step: {steps[i]}")
    plt.xlim(-3.5, 3.5)
    plt.ylim(-4., 4.75)
    plt.axis("off")
fig.tight_layout()
plt.savefig("static/reverse.png", facecolor="white")
plt.show()

VISUALIZATION OF RESULT

In [36]:
import os
import random
from utils.hand_model_lite import HandModelMJCFLite
import numpy as np
import transforms3d
import torch
import trimesh

In [66]:
mesh_path = 'C:/Users/jgual/Desktop/BEMP/Advanced Deep Learning For Robotics/ADLR_project/DexGraspNet/DexGrasp/meshdata/'
data_path = 'C:/Users/jgual/Desktop/BEMP/Advanced Deep Learning For Robotics/ADLR_project/DexGraspNet/DexGrasp/dexgraspnet/'


use_visual_mesh = False

hand_file = "mjcf/shadow_hand_vis.xml" if use_visual_mesh else "mjcf/shadow_hand_wrist_free.xml"

joint_names = [
    'robot0:FFJ3', 'robot0:FFJ2', 'robot0:FFJ1', 'robot0:FFJ0',
    'robot0:MFJ3', 'robot0:MFJ2', 'robot0:MFJ1', 'robot0:MFJ0',
    'robot0:RFJ3', 'robot0:RFJ2', 'robot0:RFJ1', 'robot0:RFJ0',
    'robot0:LFJ4', 'robot0:LFJ3', 'robot0:LFJ2', 'robot0:LFJ1', 'robot0:LFJ0',
    'robot0:THJ4', 'robot0:THJ3', 'robot0:THJ2', 'robot0:THJ1', 'robot0:THJ0'
]
translation_names = ['WRJTx', 'WRJTy', 'WRJTz']
rot_names = ['WRJRx', 'WRJRy', 'WRJRz']

In [67]:
hand_model = HandModelMJCFLite(
    hand_file,
    "mjcf/meshes")

In [73]:
grasp_code = 'core-bottle-1ae823260851f7d9ea600d1a6d9f6e07'
grasp_data = np.load(
    os.path.join(data_path, grasp_code+".npy"), allow_pickle=True)
object_mesh_origin = trimesh.load(os.path.join(
    mesh_path, grasp_code, "coacd/decomposed.obj"))

In [74]:
print(grasp_data[0]['qpos'])

{'robot0:FFJ3': -0.24548496305942535, 'robot0:FFJ2': 0.6087559461593628, 'robot0:FFJ1': 0.7188290357589722, 'robot0:FFJ0': 0.2019304633140564, 'robot0:MFJ3': -0.17577102780342102, 'robot0:MFJ2': 0.38283923268318176, 'robot0:MFJ1': 1.1127177476882935, 'robot0:MFJ0': 0.403972327709198, 'robot0:RFJ3': -0.07210764288902283, 'robot0:RFJ2': 0.1400628387928009, 'robot0:RFJ1': 0.9195994138717651, 'robot0:RFJ0': 0.41342633962631226, 'robot0:LFJ4': 0.18373392522335052, 'robot0:LFJ3': 0.025994563475251198, 'robot0:LFJ2': 0.13054056465625763, 'robot0:LFJ1': 0.9286351203918457, 'robot0:LFJ0': 0.47552821040153503, 'robot0:THJ4': -0.0618005096912384, 'robot0:THJ3': 1.1335370540618896, 'robot0:THJ2': 0.041698239743709564, 'robot0:THJ1': -0.13501165807247162, 'robot0:THJ0': -2.997061756104813e-06, 'WRJRx': 3.0913850314695135, 'WRJRy': 0.2305903394170629, 'WRJRz': -2.3657796059005416, 'WRJTx': -0.043085746467113495, 'WRJTy': 0.09048712253570557, 'WRJTz': 0.1175876185297966}


In [75]:
#index = random.randint(0, len(grasp_data) - 1)

index = 0

sample = samples[-1][0]

sample = (sample + 1)/2
sample = sample * (mean_std_max_min[2]- mean_std_max_min[3]) + mean_std_max_min[3]

qpos = grasp_data[index]['qpos']
i=0
for key in qpos:
    qpos[key] = sample[i]
    i += 1


#qpos[rot_names[0]] = 0 # X
#qpos[rot_names[1]] = 0 # Y
#qpos[rot_names[2]] = 0 # Z
rot = np.array(transforms3d.euler.euler2mat(
    *[qpos[name] for name in rot_names]))
rot = rot[:, :2].T.ravel().tolist()
hand_pose = torch.tensor([qpos[name] for name in translation_names] + rot + [qpos[name]
                         for name in joint_names], dtype=torch.float, device="cpu").unsqueeze(0)
hand_model.set_parameters(hand_pose)
hand_mesh = hand_model.get_trimesh_data(0)
object_mesh = object_mesh_origin.copy().apply_scale(grasp_data[index]["scale"])

(hand_mesh+object_mesh).show()